In [1]:
# Import necessary libraries.
import os
import glob
import imageio
import random, shutil
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
import librosa
import librosa.display

In [2]:
import requests

fname = "music.zip"
url = "https://osf.io/drjhb/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [3]:
from zipfile import ZipFile

with ZipFile(fname, 'r') as zipObj:
  # Extract all the contents of zip file in different directory
  zipObj.extractall()

In [4]:
import pandas as pd
import shutil

In [5]:
sample_path = 'Data/genres_original/jazz/jazz.00000.wav'
y, sample_rate = librosa.load(sample_path)
y_resamble= librosa.resample(y, 22050, 11025)

S = librosa.feature.melspectrogram( y_resamble[:110299] , sr=11025  )
print(S.shape)
S_DB = librosa.amplitude_to_db(S, ref=np.max)
print(S_DB.shape)

(128, 216)
(128, 216)


In [6]:
S = librosa.feature.melspectrogram( y_resamble[110299:110299*2] , sr=11025  )
print(S.shape)
S_DB = librosa.amplitude_to_db(S, ref=np.max)
print(S_DB.shape)

(128, 216)
(128, 216)


In [7]:
S = librosa.feature.melspectrogram( y_resamble[110299*2:110299*3] , sr=11025  )
print(S.shape)
S_DB = librosa.amplitude_to_db(S, ref=np.max)
print(S_DB.shape)

(128, 216)
(128, 216)


In [8]:
#time_intervals= [ [:110299] , [110299 : 110299*2] , [110299*2 :110299*3 ] ]

In [9]:
dir= 'Data/genres_original/' # the dir with all the genres
new_dir= 'new_music' # new dir for us
try:
    shutil.rmtree(new_dir, ignore_errors = False)
except:
    print('no file continue')
os.mkdir(new_dir)
genre_dict={}
for genre in os.listdir( dir ):
    genre_dir= dir+genre 
  
    os.mkdir(f'{new_dir}/{genre}')

    genre_dict[genre]= []
    print(f'we are in {genre} directory')
    use_this_dir= f'{new_dir}/{genre}' # move files here
    for music_file in os.listdir(genre_dir): # this is the music file name
        music_loc =os.path.join( genre_dir ,music_file) # the address for the music file
        
        try:
            y, sr = librosa.load(music_loc)
            # THIS IS WHERE TO START
            for i in range(3):
                if i ==0: 
                    y_resamble= librosa.resample(y[:110299], 22050, 11025) # using subsampling
                    S_C= librosa.feature.spectral_centroid( y_resamble, sr=11025  )
                    S_RO= librosa.feature.spectral_rolloff( y_resamble, sr=11025  )
                    C_STFT= librosa.feature.chroma_stft( y_resamble, sr=11025  )
                    S = librosa.feature.melspectrogram( y_resamble, sr=11025  )
                    S_DB = librosa.amplitude_to_db(S, ref=np.max)
                    genre_dict[genre].append( [S_DB, S_C , S_RO , C_STFT ] )  # tries to add the signal to the genre

                elif i ==1: 
                    y_resamble= librosa.resample(y[110299:110299*2], 22050, 11025)
                    S_C= librosa.feature.spectral_centroid( y_resamble, sr=11025  )
                    S_RO= librosa.feature.spectral_rolloff( y_resamble, sr=11025  )
                    C_STFT= librosa.feature.chroma_stft( y_resamble, sr=11025  )
                    S = librosa.feature.melspectrogram( y_resamble, sr=11025  )
                    S_DB = librosa.amplitude_to_db(S, ref=np.max)
                    genre_dict[genre].append( [S_DB, S_C , S_RO , C_STFT ] )  # tries to add the signal to the genre

                elif i==2: 
                    y_resamble= librosa.resample(y[110299*2:110299*3], 22050, 11025)
                    S_C= librosa.feature.spectral_centroid( y_resamble, sr=11025  )
                    S_RO= librosa.feature.spectral_rolloff( y_resamble, sr=11025  )
                    C_STFT= librosa.feature.chroma_stft( y_resamble, sr=11025  )
                    S = librosa.feature.melspectrogram( y_resamble, sr=11025  )
                    S_DB = librosa.amplitude_to_db(S, ref=np.max)
                    genre_dict[genre].append( [S_DB, S_C , S_RO , C_STFT ] )  # tries to add the signal to the genre


        except:
            print('error')

we are in classical directory
we are in hiphop directory
we are in pop directory
we are in country directory
we are in disco directory
we are in reggae directory
we are in jazz directory


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


error
we are in rock directory
we are in blues directory
we are in metal directory


In [10]:
# taking only 99 signals instead of 100 because we have 1 faulty file inside genre: jazz
for arr in genre_dict.keys():
    if arr == 'jazz':
        pass
    else:
        del genre_dict[arr][-4:-1]

In [11]:
len( genre_dict['pop'] )

297

In [12]:
len( genre_dict['jazz'] )

297

In [13]:
df= pd.DataFrame( genre_dict)# , dtype= np.float32 )

In [14]:
print( df.jazz[4][0].shape , 'for S_DB' )
print(df.jazz[3][1].shape  , 'for S_C' )
print(df.jazz[4][2].shape , ' for S_RO ')
print(df.jazz[5][3].shape , 'for C_STFT ')

(128, 108) for S_DB
(1, 108) for S_C
(1, 108)  for S_RO 
(12, 108) for C_STFT 


In [15]:
df_as_array= df.to_numpy()

In [18]:
#sec for audio file , first dim for class , , third for feature
print( df_as_array[200][4][0].shape , 'for S_DB' )
print(df_as_array[120][1][1].shape  , 'for S_C' )
print(df_as_array[76][3][2].shape , ' for S_RO ')
print(df_as_array[89][2][3].shape , 'for C_STFT ')

(128, 108) for S_DB
(1, 108) for S_C
(1, 108)  for S_RO 
(12, 108) for C_STFT 


In [19]:
np.save('4_features_array.npy' , df_as_array  )

# for loading the data

In [20]:
data_tmp = np.load('/content/4_features_array.npy' , allow_pickle=True)

In [21]:
data= pd.DataFrame(data_tmp)

In [23]:
# creating X for the signals
# y for the labels
y=[]
S_DB=[]
S_C= []
S_RO= []
C_STFT=[]
for label in data:
    print(label)
    for sig in data[label]:
        y.append(label)

        S_DB.append(sig[0])
        S_C.append(sig[1])
        S_RO.append(sig[2])
        C_STFT.append(sig[3])


0
1
2
3
4
5
6
7
8
9


In [24]:
data[0][0][0].shape

(128, 108)

In [25]:
tmp_arr= np.array(df_as_array)
for i in range(data.shape[0]): # files
    for m in range(10): #  labels
        for f in range(4): # features
            tmp_arr[i][m][f]=data.iloc[i][m][f][:,:108]

In [28]:
# first dim for class , sec for audio file , third for feature
print( tmp_arr[230][1][0].shape , 'for S_DB' )
print(tmp_arr[120][2][1].shape  , 'for S_C' )
print(tmp_arr[34][5][2].shape , ' for S_RO ')
print(tmp_arr[76][3][3].shape , 'for C_STFT ')

(128, 108) for S_DB
(1, 108) for S_C
(1, 108)  for S_RO 
(12, 108) for C_STFT 


In [29]:
X= np.squeeze( tmp_arr.copy() )

In [30]:
y= np.array( y )

In [31]:
len( y )

2970

## saving the features and labels

In [32]:
np.save( 'all_the_features.npy',X)
np.save('all_the_labels.npy' ,y)

# loading the data

In [33]:
x_load= np.load('all_the_features.npy', allow_pickle=True)
y_load= np.load('all_the_labels.npy', allow_pickle=True)

In [34]:
x_load[0][0][0].shape

(128, 108)

# copying 

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
!cp  '/content/all_the_features.npy' '/content/drive/MyDrive/NMA_DL_project'

In [37]:
!cp  '/content/all_the_labels.npy' '/content/drive/MyDrive/NMA_DL_project'